# Notebook 3 : Modélisation

Ce notebook a pour objectif de développer et comparer différents modèles de segmentation d'images pour identifier les objets présents dans des scènes de rue. Nous utiliserons le dataset Cityscapes, qui contient des images de haute qualité avec des annotations précises pour différents objets (voitures, piétons, bâtiments, etc.).
Nous avons précédemment explorer les données (notebook 1), puis effectuer un prétraitement (notebook 2).


## Objectifs

* Implémenter et comparer différents modèles de segmentation d'images (UNet, SegNet, etc.)
* Tester différentes fonctions de perte :
  - Entropie croisée (categorical cross-entropy)
  - Dice loss
  - Compromis entre les deux loss
* Évaluer les performances des modèles avec différentes métriques
    - Dice coefficient
    - Intersection over Union (IoU)
* Expérimenter avec l'augmentation de données pour améliorer la robustesse des modèles
* Utiliser MLflow pour le suivi des expériences et la comparaison des résultats

## Données

Le dataset Cityscapes est utilisé pour ce projet. Il contient des images de scènes de rue avec des annotations pour 8 classes d'objets :
- route (flat)
- humain (human)
- véhicule (vehicle)
- bâtiment (construction)
- objets (object)
- nature (nature)
- ciel (sky)
- vide (void)
  
## Librairies

* TensorFlow et Keras pour la construction et l'entraînement des modèles
* MLflow pour le suivi des expériences
* OpenCV pour le traitement des images
* NumPy pour les opérations numériques
* Matplotlib pour la visualisation des résultats

## Partie 1 : Configuration de l'environnement et des paramètres

In [1]:
import os
import glob
import math

import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

import mlflow
import mlflow.tensorflow
from mlflow.models import infer_signature


# Configuration des chemins
DATA_DIR = "../data/processed"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")

# Paramètres d'entraînement
NUM_CLASSES = 8
BATCH_SIZE = 4
EPOCHS = 3

## Tailles des images redimensionnées (identique au notebook 2)
IMG_HEIGHT = 128
IMG_WIDTH = 256

# Configuration de MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("segmentation_images_cityscapes")

2025-03-14 14:40:49.853385: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 14:40:49.926711: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 14:40:50.013157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741959650.124364   44309 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741959650.160029   44309 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 14:40:50.364894: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

<Experiment: artifact_location='file:///home/mehdi/Documents/OC/OC8/notebooks/mlruns/571581462687747229', creation_time=1741959660447, experiment_id='571581462687747229', last_update_time=1741959660447, lifecycle_stage='active', name='segmentation_images_cityscapes', tags={}>

## Partie 2 : Chargement et préparation des données

In [2]:
# Chargement et préparation des données

SAMPLE_SIZE = 10  # Nombre d'échantillons à utiliser pour l'entraînement et la validation




def load_data(data_dir):
    images = sorted(glob.glob(os.path.join(data_dir, "*", "*_image.png")))
    masks = sorted(glob.glob(os.path.join(data_dir, "*", "*_mask.png")))
    return images, masks


    
## chargement des données preprocésées
train_images, train_masks = load_data(TRAIN_DIR)
val_images, val_masks = load_data(VAL_DIR)
test_images, test_masks = load_data(TEST_DIR)

## on ne garde qu'un échantillon limité à SAMPLE_SIZE maximum pour les données
train_images = train_images[:SAMPLE_SIZE]
train_masks = train_masks[:SAMPLE_SIZE]
val_images = val_images[:SAMPLE_SIZE]
val_masks = val_masks[:SAMPLE_SIZE]

def data_generator(images, masks, batch_size, num_classes):
    """
    Transforme un générateur de données (images et masques) en un tf.data.Dataset.
    """

    def generator():
        """Générateur pour le tf.data.Dataset."""
        for i in range(0, len(images)):
            # Chargement de l'image
            image = cv2.imread(images[i])
            image = image / 255.0

            # Chargement du masque en grayscale
            mask = cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE)
            mask = keras.utils.to_categorical(mask, num_classes=num_classes)

            yield image, mask

    # Définir les types de données de sortie
    output_types = (tf.float32, tf.float32)

    # Définir les formes des données de sortie (None pour les dimensions variables)
    # Ici, on suppose que toutes les images ont la même taille
    # Si les images ont des tailles variables, il faudra adapter cette partie
    image_shape = cv2.imread(images[0]).shape
    mask_shape = (image_shape[0], image_shape[1], num_classes)

    output_shapes = (tf.TensorShape(image_shape), tf.TensorShape(mask_shape))

    # Créer le tf.data.Dataset à partir du générateur
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_types=output_types,
        output_shapes=output_shapes
    )

    # Batching
    dataset = dataset.batch(batch_size)

    return dataset



## Partie 3 : Définition des modèles

UNet mini

In [3]:
def build_unet_mini(img_height, img_width, num_classes):
    inputs = keras.layers.Input(shape=(img_height, img_width, 3))
    
    # Downsampling
    conv1 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)
    
    conv2 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)
    
    # Bottleneck
    conv3 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    
    # Upsampling
    up4 = keras.layers.UpSampling2D((2, 2))(conv3)
    merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
    conv4 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(merge4)
    
    up5 = keras.layers.UpSampling2D((2, 2))(conv4)
    merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
    conv5 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(merge5)
    
    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)
    
    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model

Modèle UNet

In [4]:
def build_unet(img_height, img_width, num_classes):
    """
    Définition du modèle UNET spécifique à la segmentation d'images.
    U-Net : https://fr.wikipedia.org/wiki/U-Net
    """

    ## Entrée
    inputs = keras.layers.Input(shape=(img_height, img_width, 3))

    ## Bloc 1
    conv1 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)

    ## Bloc 2
    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)

    ## Bloc 3
    conv3 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    ## Bloc 4
    up4 = keras.layers.UpSampling2D((2, 2))(conv3)
    merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
    conv4 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge4)

    ## Bloc 5
    up5 = keras.layers.UpSampling2D((2, 2))(conv4)
    merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
    conv5 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge5)

    ## Couche de sortie
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)
    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model

Modèle VGG16

In [5]:
def build_vgg16_unet(img_height, img_width, num_classes):
    """
    Construit un modèle VGG16-UNet avec des couches d'upsampling supplémentaires
    pour s'assurer que la taille de la sortie correspond à la taille de l'entrée.
    """
    # Charger le modèle VGG16 pré-entraîné sans la partie classification
    vgg16 = keras.applications.VGG16(input_shape=(img_height, img_width, 3),
                                     include_top=False,
                                     weights='imagenet')

    # Encoder (VGG16)
    vgg16_output = vgg16.output

    # Decoder (UNet-like)
    # Ajouter des couches Conv2DTranspose supplémentaires pour augmenter la taille
    up0 = keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(vgg16_output) # Taille: H/16 * 2, W/16 * 2
    merge0 = keras.layers.concatenate([vgg16.get_layer('block5_conv3').output, up0], axis=-1)
    conv0 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(merge0)

    up1 = keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv0) # Taille: H/8 * 2, W/8 * 2
    merge1 = keras.layers.concatenate([vgg16.get_layer('block4_conv3').output, up1], axis=-1)
    conv1 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(merge1)

    up2 = keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv1) # Taille: H/4 * 2, W/4 * 2
    merge2 = keras.layers.concatenate([vgg16.get_layer('block3_conv3').output, up2], axis=-1)
    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge2)

    up3 = keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2) # Taille: H/2 * 2, W/2 * 2
    merge3 = keras.layers.concatenate([vgg16.get_layer('block2_conv2').output, up3], axis=-1)
    conv3 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge3)

    up4 = keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv3) # Taille: H * 2, W * 2
    merge4 = keras.layers.concatenate([vgg16.get_layer('block1_conv2').output, up4], axis=-1)
    conv4 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(merge4)

    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv4)

    model = keras.models.Model(inputs=vgg16.input, outputs=outputs)
    return model

MobileNetV2_Unet

In [6]:
def build_mobilenetv2_unet(img_height, img_width, num_classes):
    """Construit un modèle MobileNetV2-UNet."""
    # Charger le modèle MobileNetV2 pré-entraîné sans la partie classification
    mobilenetv2 = keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                     include_top=False,
                                     weights='imagenet')

    # Récupérer la sortie de MobileNetV2
    mobilenet_output = mobilenetv2.output

    # Reshape de la sortie de MobileNetV2
    reshape = keras.layers.Reshape((img_height // 32, img_width // 32, mobilenet_output.shape[-1]))(mobilenet_output)

    # Decoder (UNet-like) avec 5 couches Conv2DTranspose
    up1 = keras.layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(reshape) # 8x16
    conv1 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(up1)

    up2 = keras.layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(conv1) # 16x32
    conv2 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up2)

    up3 = keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(conv2) # 32x64
    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up3)

    up4 = keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(conv3) # 64x128
    conv4 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up4)

    up5 = keras.layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(conv4) # 128x256
    conv5 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up5)

    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

    model = keras.models.Model(inputs=mobilenetv2.input, outputs=outputs)
    return model


## Partie 4 : Définition des fonctions de perte et des métriques

In [7]:

## (1) Métriques

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Calcule le coefficient de Dice, une métrique de similarité entre deux ensembles.
    Il est souvent utilisé pour évaluer la performance des modèles de segmentation.

    Args:
        y_true (Tensor): Les valeurs de vérité terrain (ground truth).
        y_pred (Tensor): Les prédictions du modèle.
        smooth (float, optional): Un terme de lissage pour éviter la division par zéro. Defaults to 1e-6.

    Returns:
        float: Le coefficient de Dice, une valeur entre 0 et 1 inclusivement.

    Plage de valeur :
        - 0 : Indique une absence totale de chevauchement entre les prédictions et la vérité terrain.
        - 1 : Indique un chevauchement parfait entre les prédictions et la vérité terrain (les ensembles sont identiques).
    """
    y_true_f = keras.backend.flatten(tf.cast(y_true, tf.float32)) # Conversion de y_true en float32
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) + smooth)

def iou_metric(y_true, y_pred, smooth=1e-6):
    """
    Calcule l'IoU (Intersection over Union), également appelé indice de Jaccard,
    une métrique couramment utilisée pour évaluer la performance des modèles de segmentation.

    Args:
        y_true (Tensor): Les valeurs de vérité terrain (ground truth).
        y_pred (Tensor): Les prédictions du modèle.
        smooth (float, optional): Un terme de lissage pour éviter la division par zéro. Defaults to 1e-6.

    Returns:
        float: L'IoU, une valeur entre 0 et 1 inclusivement.

    Plage de valeur :
        - 0 : Indique une absence totale de chevauchement entre les prédictions et la vérité terrain.
        - 1 : Indique un chevauchement parfait entre les prédictions et la vérité terrain.
    """
    y_true_f = keras.backend.flatten(tf.cast(y_true, tf.float32)) # Conversion de y_true en float32
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    union = keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


## (2) Fonctions de perte

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def mixed_loss(y_true, y_pred):
    """Fonction de perte combinant categorical crossentropy et dice loss."""
    return 0.5 * keras.losses.CategoricalCrossentropy()(y_true, y_pred) + 0.5 * dice_loss(y_true, y_pred)

## Partie 5 : Augmentation des données (Data Augmentation)

On génère des données "augmentées", à savoir en faisant des modifications légères (retournement, rotation, zoom...)

In [8]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1)
])

def augmented_data_generator(images, masks, batch_size, num_classes):
    """
    Crée un générateur de données augmentées en utilisant tf.data.Dataset.
    """

    def augment(image, mask):
        """Fonction pour appliquer l'augmentation de données."""
        augmented_image = data_augmentation(image)
        return augmented_image, mask

    # Créer le dataset à partir du générateur de données de base
    dataset = data_generator(images, masks, batch_size, num_classes)

    # Appliquer l'augmentation de données
    dataset = dataset.map(augment)

    return dataset



2025-03-14 14:41:00.749356: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## Partie 6 : Entraînement des modèles

### Configuration des modèles

In [9]:
models = {
    "UNet_mini": build_unet_mini(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES),
    "UNet_base": build_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES),
    "VGG16_UNet": build_vgg16_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES),
    "MobileNetV2_pretrained": build_mobilenetv2_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES)
}


2025-03-14 14:41:01.682349: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18874368 exceeds 10% of free system memory.
2025-03-14 14:41:01.712001: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18874368 exceeds 10% of free system memory.
2025-03-14 14:41:01.723432: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18874368 exceeds 10% of free system memory.
/tmp/ipykernel_44309/1192823791.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenetv2 = keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
2025-03-14 14:41:04.023168: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23592960 exceeds 10% of free system memory.
2025-03-14 14:41:04.057912: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation 

### Configuration des loss (fonctions de perte)

In [10]:

losses = {
    "categorical_crossentropy": keras.losses.CategoricalCrossentropy(),
    "dice_loss": dice_loss,
    "mixed_loss": mixed_loss
}

### Entraînement des modèles


Entraînement **sans** data augmentation

In [ ]:


# Calculer steps_per_epoch
steps_per_epoch = math.ceil(len(train_images) / BATCH_SIZE)

# Calculer validation_steps
validation_steps = math.ceil(len(val_images) / BATCH_SIZE)

# Créer un exemple d'entrée
input_example = np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3).astype(np.float32)


## On itère sur les modèles
for model_name, model in models.items():
    ## On itère (2e boucle) sur les loss
    for loss_name, loss in losses.items():
        ## on sauve les résultats dans MFlow (model + loss)
        with mlflow.start_run(run_name=f"{model_name}_{loss_name}"):

            
            ## génération des données d'entraînement
            train_generator = data_generator(train_images,
                                 train_masks,
                                 BATCH_SIZE,
                                 NUM_CLASSES)

            ## génération des données de validation
            val_generator = data_generator(val_images,
                               val_masks,
                               BATCH_SIZE,
                               NUM_CLASSES)


            ## Complilation
            model.compile(optimizer='adam', loss=loss, metrics=[iou_metric])

            ## Fitting

            
            # Obtenir un exemple de sortie (prédiction)
            output_example = model.predict(input_example)

            # Déduire la signature
            signature = infer_signature(input_example, output_example)
            
            print(f"fitting {model_name}_{loss_name}")

            #mlflow.keras.autolog()
            model.fit(train_generator,
                      validation_data=val_generator,
                      epochs=EPOCHS,
                     steps_per_epoch=steps_per_epoch,
                     validation_steps = validation_steps)

            # log model

            mlflow.keras.log_model(model,
                                   "model",
                                   registered_model_name=f"{model_name}_{loss_name}",
                                   signature=signature,
                                   pip_requirements=["tensorflow", "keras", "opencv-python"])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
fitting UNet_mini_categorical_crossentropy
Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 961ms/step - iou_metric: 0.0665 - loss: 2.0851 - val_iou_metric: 0.0700 - val_loss: 2.0382
Epoch 2/3


2025-03-14 14:41:11.251346: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/mehdi/Documents/OC/OC8/.venv/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - iou_metric: 0.0000e+00 - loss: 0.0000e+00 - val_iou_metric: 0.0700 - val_loss: 2.0382
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 810ms/step - iou_metric: 0.0719 - loss: 2.0169 - val_iou_metric: 0.0782 - val_loss: 1.9744


Successfully registered model 'UNet_mini_categorical_crossentropy'.
Created version '1' of model 'UNet_mini_categorical_crossentropy'.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
fitting UNet_mini_dice_loss
Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 712ms/step - iou_metric: 0.0833 - loss: 0.8475 - val_iou_metric: 0.1029 - val_loss: 0.8126
Epoch 2/3


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
fitting MobileNetV2_pretrained_categorical_crossentropy
Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - iou_metric: 0.0749 - loss: 2.0480 - val_iou_metric: 0.2611 - val_loss: 13.0621
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step - iou_metric: 0.0000e+00 - loss: 0.0000e+00 - val_iou_metric: 0.2611 - val_loss: 13.0621
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - iou_metric: 0.2105 - loss: 5.2202 - val_iou_metric: 0.0889 - val_loss: 1.9526


Successfully registered model 'MobileNetV2_pretrained_categorical_crossentropy'.
Created version '1' of model 'MobileNetV2_pretrained_categorical_crossentropy'.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
fitting MobileNetV2_pretrained_dice_loss
Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - iou_metric: 0.1807 - loss: 0.7187 - val_iou_metric: 0.2784 - val_loss: 0.5661
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 374ms/step - iou_metric: 0.0000e+00 - loss: 0.0000e+00 - val_iou_metric: 0.2784 - val_loss: 0.5661
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - iou_metric: 0.2985 - loss: 0.5418 - val_iou_metric: 0.2737 - val_loss: 0.5717


Successfully registered model 'MobileNetV2_pretrained_dice_loss'.
Created version '1' of model 'MobileNetV2_pretrained_dice_loss'.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
fitting MobileNetV2_pretrained_mixed_loss
Epoch 1/3
1/3 ━━━━━━━━━━━━━━━━━━━━ 1:28 44s/step - iou_metric: 0.3042 - loss: 23.6420

Entraînement **avec** data augmentation 

In [ ]:
# Entraînement avec augmentation de données
for model_name, model in models.items():
    for loss_name, loss in losses.items():
        with mlflow.start_run(run_name=f"{model_name}_{loss_name}_augmented"):

            ## génération des données d'entraînement augmentées
            augmented_train_generator = augmented_data_generator(train_images,
                                                     train_masks,
                                                     BATCH_SIZE,
                                                     NUM_CLASSES)
            ## génération des données de validation
            val_generator = data_generator(val_images,
                               val_masks,
                               BATCH_SIZE,
                               NUM_CLASSES)

            
            model.compile(optimizer='adam', loss=loss, metrics=[iou_metric])
            
            print(f"fitting {model_name}_{loss_name}_augmented")
            
            model.fit(augmented_train_generator,
                      validation_data=val_generator,
                      epochs=EPOCHS)

            # log model
            mlflow.keras.log_model(model,
                                   "model",
                                   registered_model_name=f"{model_name}_{loss_name}_augmented")

## Partie 7 : Évaluation des modèles

In [ ]:
# Évaluation des modèles entraînés
for model_name, model in models.items():
    for loss_name, loss in losses.items():


        ## génération des données de test
        test_generator = data_generator(test_images,test_masks,
                                                     BATCH_SIZE,
                                                     NUM_CLASSES)
        # Chargement du modèle entraîné
        model_path = f"mlruns/0/{mlflow.search_runs(filter_string=f'tags.mlflow.runName = \'{model_name}_{loss_name}\'').iloc[0].run_id}/artifacts/model/data/model.keras"
        loaded_model = keras.models.load_model(model_path,
                                               custom_objects={'iou_metric': iou_metric, 'dice_loss': dice_loss})

        # Évaluation du modèle sur l'ensemble de test
        loss, iou = loaded_model.evaluate(test_generator)
        print(f"Modèle {model_name} avec perte {loss_name} : Loss = {loss}, IoU = {iou}")

# Évaluation des modèles entraînés avec augmentation de données
for model_name, model in models.items():
    for loss_name, loss in losses.items():

        ## génération des données de test
        test_generator = data_generator(test_images,test_masks,
                                                     BATCH_SIZE,
                                                     NUM_CLASSES)
        
        # Chargement du modèle entraîné
        model_path = f"mlruns/0/{mlflow.search_runs(filter_string=f'tags.mlflow.runName = \'{model_name}_{loss_name}_augmented\'').iloc[0].run_id}/artifacts/model/data/model.keras"
        loaded_model = keras.models.load_model(model_path, custom_objects={'iou_metric': iou_metric, 'dice_loss': dice_loss})

        # Évaluation du modèle sur l'ensemble de test
        loss, iou = loaded_model.evaluate(test_generator)
        print(f"Modèle {model_name} avec perte {loss_name} et augmentation : Loss = {loss}, IoU = {iou}")


## Conclusion

Ce notebook a exploré différentes approches pour la segmentation d'images de scènes de rue. Les résultats montrent que l'augmentation de données peut améliorer les performances des modèles. 

### Améliorations possibles

* Tester d'autres modèles de segmentation, tels que SegNet ou DeepLabv3.
* Expérimenter avec différentes fonctions de perte et métriques.
* Utiliser des techniques d'augmentation de données plus avancées.
* Optimiser les hyperparamètres des modèles.